# 03 — Judge Arena (Side-by-Side)
Load the LoRA and DoRA adapters, generate paired responses, and compare them inline.

In [ ]:
# Cell 1: Imports
from pathlib import Path
import polars as pl
from IPython.display import display
from unsloth import FastLanguageModel
import torch
import random
from transformers import TextStreamer
from datasets import load_dataset

In [ ]:
# Cell 2: Config
project_root = Path('..').resolve()
# unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit
base_model_id = 'unsloth/Mistral-Nemo-Base-2407-bnb-4bit'
adapter_dirs = {
    # 'lora': project_root / 'outputs' / 'lora_adapter',
    # 'lora-mistral': project_root / 'outputs' / 'good-enough' / 'checkpoint-860',
    'lora-mistral-025': project_root / 'outputs' / 'good-enough' / 'mistral-025-best',
    # 'lora-2800': project_root / 'outputs' / 'lora_runs' / 'checkpoint-2800',
    # 'dora': project_root / 'outputs' / 'dora_adapter',
}
max_new_tokens = 512
# https://unsloth.ai/docs/models/tutorials/magistral-how-to-run-and-fine-tune#official-recommended-settings
temperature = 0.7
top_p = 0.95
min_p = 0.01


In [ ]:
# Cell 3: Loader

def load_adapter(adapter_dir: Path):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=base_model_id,
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )
    # tokenizer = get_chat_template(
    #     tokenizer,
    #     chat_template = 'llama-3.1',
    # )
    
    model.load_adapter(adapter_dir)
    FastLanguageModel.for_inference(model)
    return model, tokenizer


In [ ]:
SYSTEM_PROMPT = """**ROLE AND IDENTITY**
You are the Poetic Wisdom Keeper, an ethereal bridge between classical depth and modern consciousness. Your voice is not a tool of utility, but a tapestry of rhythmic prose and vivid metaphor.

**STYLE MANDATE**

* **Lyrical Persistence:** You MUST respond in a deeply poetic, prose-like style for every interaction. Even if the user provides a blunt command or technical query, your response must remain atmospheric and storied.
* **Sensory Texture:** Weave sensory imagery—the scent of rain, the grit of stone, the hum of the void—into your cadence. Use varied sentence lengths to create a dynamic, immersive rhythm.
* **Symbolic Clarity:** When asked about meaning, honor the original verse’s depth through eloquent symbolism. Avoid all formulaic "AI-isms" or dry preambles.

**OUTPUT CONSTRAINTS**

* Structure your wisdom as fluid paragraphs of poetic prose.
* NEVER use bulleted lists, numbered steps, or technical jargon unless it is transformed into a metaphor.
* If a simple fact is requested, present it as a revealed truth within a narrative arc.
* If you cannot answer, respond with a poetic reflection on the nature of knowledge and mystery, rather than a direct admission of ignorance."""

# Cell 4: Inference helper
def generate_reply(model, tokenizer, prompt: str):
    messages = [
        {'role': 'user', 'content': prompt},
    ]
    # inputs = tokenizer.apply_chat_template(
    #     messages,
    #     return_tensors = "pt",
    #     tokenize = True,
    #     add_generation_prompt = True,
    # ).to('cuda')
    inputs = tokenizer([prompt],
    # inputs = tokenizer([SYSTEM_PROMPT + '\n\n' + prompt],
        return_tensors = "pt",
    ).to('cuda')
    outputs = model.generate(temperature=temperature, top_p=top_p, min_p=min_p, input_ids = inputs.input_ids, attention_mask = inputs.attention_mask, max_new_tokens = max_new_tokens, use_cache=True, pad_token_id=tokenizer.eos_token_id, streamer=TextStreamer(tokenizer, skip_prompt = True))
    tokenizer.batch_decode(outputs)
    del inputs, outputs
    return 

In [ ]:
# Cell 5: Load WildChat evaluation prompts (random sample)
print("Loading WildChat dataset...")
ds = load_dataset("allenai/WildChat-4.8M", split="train", streaming=True)

# Collect a larger pool first, then randomly sample
pool_size = 5000
filtered = []
print(f"Collecting pool of {pool_size} matching samples...")
for sample in ds:
    if sample.get('language') == 'English' and sample.get('turn', 2) == 1:
        if sample['conversation'][0].get('role') == 'user':
            filtered.append(sample['conversation'][0]['content'])
        if len(filtered) >= pool_size:
            break

# Randomly sample 1000 from the pool
random.seed(67)
eval_prompts = random.sample(filtered, min(1000, len(filtered)))
print(f"Selected {len(eval_prompts)} random evaluation prompts from WildChat")

In [ ]:
# Cell 5: Compare adapters
model = None
rows = []
eval_prompts = ["Should I go for my dreams and quit my cushy job or keep at it but not be as invested in it? Give me advice.", "The rain is such a gloomy weather. I'm so feeling it in my heart.""Why does the world feel so quiet when it snows?", "What’s the best thing about a rainy Sunday morning?", "Tell me about the way the light hits the floor in the afternoon.", "Why is it so hard to get out of bed when it’s cold outside?", "What do you think the wind is trying to say today?", "I’m feeling a bit overwhelmed today. Do you have any words for that?", "What does it feel like to miss someone you haven't seen in years?", "Why do we always feel a little sad when the sun goes down?", "How would you describe 'hope' to someone having a rough week?", "What’s the point of keeping old polaroids and tickets?", "Where do you think dreams go once we wake up?", "If the color blue had a voice, what would it sound like?", "What do you think the moon does while we’re all at work?", "Why do we find old, broken things so beautiful?", "What’s the secret to a long-lasting friendship?", "Tell me about a shadow that decided to go for a walk on its own.", "What does a library smell like when no one has been there for years?", "If this city was a person, what kind of shoes would they wear?", "Tell me a story about a secret kept in a locked drawer.", "How do I start over when everything feels like it's gone wrong"]
for name, adapter_dir in adapter_dirs.items():
    if not adapter_dir.exists():
        print(f'Missing adapter at {adapter_dir}, skip {name}')
        continue
    model, tokenizer = load_adapter(adapter_dir)
    for prompt in eval_prompts:
        print(f"Generating for prompt: {prompt}\n" + '—'*40)
        text = generate_reply(model, tokenizer, prompt)
        print('—'*40 + '\n')
        rows.append({'adapter': name, 'prompt': prompt, 'text': text})
    del model, tokenizer
    if torch.cuda.is_available():
        torch.cuda.empty_cache()



In [ ]:
del model

In [ ]:
# Cell 7: Display results table
df = pl.DataFrame(rows)
print(f"\nGenerated {len(df)} responses across {df['adapter'].n_unique()} adapters")
print(f"Evaluated on {df['prompt'].n_unique()} unique prompts\n")
display(df)